In [325]:
%whos

Variable               Type             Data/Info
-------------------------------------------------
Feat_Fin               DataFrame                       max_id    <...>n\n[104 rows x 7 columns]
Feature_list_lengths   list             n=0
Json_obj               str              ['B:MUTA:COMP:ERBB2::', '<...>732:USP39:TssD261:Shore']
ax                     AxesSubplot      AxesSubplot(0.125,0.125;0.775x0.755)
bool_count             Series           0       True\n1       Tru<...> Length: 104, dtype: bool
c                      Series           3     ["B:MUTA:LOF:CDK6::<...>_feat_list, dtype: object
c_typ                  str              UVM
cncr_typs              list             n=26
data                   list             n=1
df                     DataFrame                                 <...>\n[104 rows x 23 columns]
f                      Figure           Figure(432x648)
fbd_filt               DataFrame                                 <...>n\n[79 rows x 23 columns]
fbed_count    

In [1]:
pwd

'/Users/karlberb/work/tmp/v8/hc'

In [1]:
#feat_calc_pred
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt

In [2]:
feat=pd.read_csv('/Users/karlberb/work/tmp/v8/Results/feat_3_col/fbed_rfe_feats.tsv')

In [6]:
### .  ### Run for stacker
cncr_typs=['ACC','BLCA','BRCA','CESC','COADREAD',
            'ESCC','GEA','HNSC','KIRCKICH','KIRP',
            'LGGGBM','LIHCCHOL','LUAD','LUSC','MESO',
            'OV','PAAD','PCPG','PRAD','SARC',
            'SKCM','TGCT','THCA','THYM','UCEC',
            'UVM']

In [16]:
### Run 1
# features=['fbed']
store_df2=pd.DataFrame() #Off for rfe run

### Run 2
features=['rfe']

#clfs=['ada'] # Devel
clfs=['ada','bnb','dt','et','gnb','knn','logreg','pa','rf','sgd','svm']
cncr_typs=['ACC','BLCA','BRCA','CESC','COADREAD',
            'ESCC','GEA','HNSC','KIRCKICH','KIRP',
            'LGGGBM','LIHCCHOL','LUAD','LUSC','MESO',
            'OV','PAAD','PCPG','PRAD','SARC',
            'SKCM','TGCT','THCA','THYM','UCEC',
            'UVM']
            
# Toggle on creation for fbed run 
# Toggle off rfe append run
#

###

for i in features:
    for j in clfs: # gausian process missing
        for s, k in enumerate(cncr_typs):
            #print(s)
            #print(k)
            
            # Feature toggle for individual runs
            #q=rfe_ext.iloc[s,2]
#             q=feat.iloc[s,2]
#             r=json.loads(q)
#             ft_ct=len(r)
            # I think the above carries from the plotting
            
#   ******    # Run 1
            #file=pd.read_csv('/Users/karlberb/work/tmp/v8/Results/reprocess/'+i+'_'+j+'/'+k+'_'+i+'eBIC_'+j+'_predictions_20200331_bk.tsv')
            
            #Run 2
            file=pd.read_csv('/Users/karlberb/work/tmp/v8/Results/reprocess/'+i+'_'+j+'/'+k+'_'+i+'_'+j+'_predictions_20200331_bk.tsv')
            
            #### .    ######
   ######         
            file=file[(file['Test']==1)] # Filter for test results # Critical #
   ###### 
            ####      ######
            
            file.columns=['Sample_ID','Repeat','Fold','Test','Label','Prediction']

            string=k+':'
            file.Label=file.Label.str.replace(string,'')
            file.Prediction=file.Prediction.str.replace(string,'')
            file=file.sort_values(by=['Label','Prediction'])
            
            # Get number of features
            sub_n_mx=file.Label.max()
            sm=int(sub_n_mx)
            
            # Make iterable len of number of features
            sm_plus1=sm+1
            Y=np.arange(1,sm_plus1)
            X=list(range(1,sm_plus1))
            g_list=[]
            data=[0]

            # For each cancer with X subtypes:
            for l in X:
                m=str(l)
                grnd_bool=file['Label']==m
                grnd_full=file[grnd_bool]
                gn=len(grnd_full)
                g_list=g_list+[gn]
                # Prediction lists
                p_list=[]
                # Make one for each g list
                for n in Y:
                    o=str(n)
                    p_bool=grnd_full['Prediction']==o
                    p_full=grnd_full[p_bool]
                    p_len=len(p_full)
                    p_list=p_list+[p_len]
                    #print(p_len)
                data.append(p_list)
            data[0]=g_list
            
            # Begin analysis section step one, make 572 x 7
            store=[] # This is a horizontal df
            #Columns will be 572 combos
            #Rows are subtypes, n=range(2,7)
            for zer_indx, g_star in enumerate(g_list):
                one_indx=zer_indx+1
                p_lst=data[one_indx]
                p_star=p_lst[zer_indx]
                raw_acc=p_star/g_star
                name=i+'_'+j+'_'+k
                store.append(raw_acc)
                dat={name:store}
                temp_df=pd.DataFrame(dat)
            store_df2=pd.concat([store_df2, temp_df], axis=1)
            #Create store_df for fbed run, then toggle off
        

In [18]:
store_df2.to_csv('rfe_raw_sums.csv')

In [13]:
pwd

'/Users/karlberb/work/tmp/v8/hc'

In [22]:
store_572 = pd.concat([store_df, store_df2], axis=1)

In [24]:
store_572.to_csv('hybrid_raw_sums.csv')

In [3]:
### .  ### Start here with file import
hyb = pd.read_csv('hybrid_raw_sums.csv')

In [ ]:
# Do calc on range of classifer differences

In [8]:
### .  ### STACKER - use on store_572

## This chunk stacks up the columns and deletes the NaNs

# c_typ is cancer type from 572 columns headers
#ct=store_df.filter(regex='ACC') #devel ?
Feat_Fin=pd.DataFrame(columns=["max_id","score"]) #add fbed score colmn
for ii in cncr_typs: # ii is ACC, BLCA, etc
    c_typ=hyb.filter(regex=ii) # make canc typ spec df, 22 combo cols
    # Should have same number subtype prediction rows
    
    for row in c_typ.index: # Look through the rows
        sub_row=c_typ.iloc[row,:] # Extract all columns for each row
        #print(sub_row) #series data type - important inspection step
        
        max_score=max(sub_row) # Pull the winning score
        
        max_id=sub_row.idxmax() # Pull corresponding ID, look up
        data=[{'max_id':max_id,'score':max_score}]
        tmp=pd.DataFrame(data,columns=["max_id","score"])
        Feat_Fin=pd.concat([Feat_Fin,tmp])
Feat_Fin=Feat_Fin.dropna()
print(Feat_Fin.head())

         max_id     score
0   fbed_rf_ACC  0.991818
0   fbed_rf_ACC  0.930417
0  fbed_gnb_ACC  0.852105
0   rfe_rf_BLCA  0.972230
0  rfe_svm_BLCA  0.919841
